```cypher
CREATE CONSTRAINT ON (p:Paciente) ASSERT p.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Rastreavel) ASSERT r.id IS UNIQUE;
CREATE CONSTRAINT ON (r:Rastreavel) ASSERT EXISTS(r.tipo);
CREATE CONSTRAINT ON (r:Rastreavel) ASSERT EXISTS(r.nome);
CREATE CONSTRAINT ON ()-[c:CHECKIN]-() ASSERT EXISTS(c.date);

LOAD CSV WITH HEADERS FROM "https://media.githubusercontent.com/media/TiagodePAlves/mc536-projeto/master/dados/flaredown.csv" AS line
WITH line
WHERE line.user_id IS NOT NULL AND line.trackable_id IS NOT NULL
MERGE (p:Paciente {id: line.user_id})
	ON CREATE SET
    	(CASE WHEN line.age IS NOT NULL THEN p END).idade = toInteger(line.age),
    	(CASE WHEN line.sex IS NOT NULL THEN p END).sexo = line.sex,
    	(CASE WHEN line.country IS NOT NULL THEN p END).pais = line.country
MERGE (r:Rastreavel {id: toInteger(line.trackable_id)})
	ON CREATE SET
    	r.tipo = line.trackable_type,
    	r.nome = line.trackable_name
CREATE (p)-[c:CHECKIN {date: datetime(line.checkin_date)}]->(r)
SET (CASE WHEN line.line.trackable_value IS NOT NULL THEN c END).value = line.trackable_value;
```

```cypher
CREATE CONSTRAINT ON ()-[o:OCORRE_COM]-() ASSERT EXISTS(o.em);

MATCH (r1:Rastreavel)<-[:CHECKIN]-(p:Paciente)-[:CHECKIN]->(r2:Rastreavel)
WHERE r1 <> r2
CREATE (r1)-[:OCORRE_COM {em: p.id}]->(r2);
```